In [ ]:
import numpy as np
import pandas as pd

Cleaning train dataset

In [ ]:
train_df = pd.read_csv("./twitter_training.csv")
absurd = train_df.columns[3]

train_df.rename(columns = {
    'Positive' : 'Label',
    absurd : 'text'
}, inplace=True)

train_df.drop(columns=['Borderlands', '2401'], inplace=True)
train_df.head()

In [ ]:
train_df['Label'].value_counts()

In [ ]:
print(f"Null Rows:\n{train_df.isnull().sum()}\n")
print(f"Duplicated Rows: {int(train_df.duplicated().sum())}")


In [ ]:
print("Percentage of Empty and duplicated rows: ", round(
    int(train_df.isnull().sum()['text'] + train_df.duplicated().sum()) * 100 / len(train_df),
    3), "%")


#### Since the empty and duplicated needs to be delt with, we shall drop it

In [ ]:
train_df.dropna(inplace=True)
train_df.drop_duplicates(inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style = 'darkgrid')
sns.countplot(data = train_df, x=train_df['Label'], hue=train_df['Label'], palette='magma')
plt.title('Label Count')
plt.ylabel('Count')
plt.xlabel('Labels')
plt.show()

#### Encoding the Labels

In [ ]:
train_df['Sentiment'] = train_df['Label'].replace({
    'Irrelevant' : 0,
    'Neutral' : 1,
    'Negative' : 2,
    'Positive' : 3 
})
train_df.drop(columns='Label', inplace=True)
train_df.head()

Cleaning validation dataset

In [ ]:
val_df = pd.read_csv('./twitter_validation.csv')
absurd2 = val_df.columns[3]
val_df.rename(columns = {
    'Irrelevant' : 'Label',
    absurd2 : 'text'
}, inplace=True)
val_df.drop(columns=['Facebook', '3364'], inplace=True)
val_df['Sentiment'] = val_df['Label'].replace({
    'Irrelevant' : 0,
    'Neutral' : 1,
    'Negative' : 2,
    'Positive' : 3 
})
val_df.drop(columns='Label', inplace=True)
val_df.head()

In [ ]:
val_df.duplicated().sum()

In [ ]:
val_df.isnull().sum()

In [ ]:
val_df.shape

# MODEL TIME!!!

In [ ]:
# Import necessary libraries
from transformers import TFBertForSequenceClassification, BertTokenizerFast, AdamWeightDecay
import tensorflow as tf

In [ ]:
size = 8

In [ ]:
# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
from transformers import BertTokenizerFast
import tensorflow as tf

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Tokenize texts
train_encodings = tokenizer(list(train_df["text"]), truncation=True, padding=True)
val_encodings = tokenizer(list(val_df["text"]), truncation=True, padding=True)

# Convert to tf.data.Dataset
train_df = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"]
    },
    list(train_df["Sentiment"])
)).batch(size)

val_df = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": val_encodings["input_ids"],
        "attention_mask": val_encodings["attention_mask"]
    },
    list(val_df["Sentiment"])
)).batch(size)


In [ ]:
# Load the model
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# Use AdamWeightDecay optimizer
optimizer = AdamWeightDecay(learning_rate=3e-5, weight_decay_rate=0.01)

# Loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])


In [ ]:
# Train the model
model.fit(
    train_df,
    validation_data=val_df,
    epochs=3
)

In [ ]:
# Assuming `val_df` is your validation dataset
# Evaluate the model
loss, accuracy = model.evaluate(val_df)

print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

In [ ]:
model.save('./Analyser')